In [ ]:
import os

# Replace "your_username" and "your_api_key" with your actual Kaggle credentials
kaggle_credentials = '''{"username":"KULLANICI ADI","key":"API ANAHTARI"}''' # Profil -> Ayarlar -> API key oluştur. Sonrasında json dosyasındaki satırı buraya yapıştır. Kaggle notebook üzerinde çalıştırmak için gerekli. Localde çalıştırıyorsanız indirme işlemlerini atlayabilirsiniz.

# Create the required directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Write the kaggle.json file
with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(kaggle_credentials)

# Set the proper permissions
os.chmod("/root/.kaggle/kaggle.json", 600)

print("kaggle.json successfully created and configured!")


kaggle.json successfully created and configured!


In [7]:
!kaggle competitions list

ref                                                                                deadline             category                reward  teamCount  userHasEntered  
---------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2      2025-04-01 23:59:00  Featured         2,117,152 Usd        706           False  
https://www.kaggle.com/competitions/gemma-language-tuning                          2025-01-15 00:59:00  Analytics          150,000 Usd          0           False  
https://www.kaggle.com/competitions/jane-street-real-time-market-data-forecasting  2025-01-13 23:59:00  Featured           120,000 Usd       2218           False  
https://www.kaggle.com/competitions/nfl-big-data-bowl-2025                         2025-01-08 23:59:00  Analytics          100,000 Usd          0           False  
https://www.kagg

In [9]:
!kaggle competitions download -c blg-454-e-fall-2024-term-project

blg-454-e-fall-2024-term-project.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip -o blg-454-e-fall-2024-term-project.zip -d /kaggle/working

Archive:  blg-454-e-fall-2024-term-project.zip
  inflating: /kaggle/working/sampleSubmission.csv  
  inflating: /kaggle/working/train_feats.npy  
  inflating: /kaggle/working/train_labels.csv  
  inflating: /kaggle/working/valtest_feats.npy  


In [10]:
rm blg-454-e-fall-2024-term-project.zip

# SAVE the version HERE AFTER UPLOAD the files on /kaggle/working to keep files on notebook.(they would be removed after leaving.)

In [33]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Step 1: Load the Training Data (features and labels)
# Assuming your .npy files are in the current working directory and each .npy file has 4 features.
X_train = np.load('train_feats.npy',allow_pickle=True).item()  # Shape: (40000, 4, 512)

# Load the labels from the CSV file
labels_df = pd.read_csv('train_labels.csv')  # Assuming two columns: 'id' and 'label'
y_train = labels_df['label'].values  # Labels for training data

In [35]:
# Check the type of the first element in the loaded data
print(type(X_train))  # Should print <class 'numpy.ndarray'>
# Inspect the first few elements to see if they are lists or arrays
print(X_train)  # Should print the first item (should be a list or array with 4 features)


<class 'dict'>
{'idx': array([    0,     1,     2, ..., 39997, 39998, 39999], dtype=int32), 'resnet_feature': array([[0.882933  , 1.7576884 , 0.03708305, ..., 3.1606352 , 0.0134144 ,
        0.6481762 ],
       [1.856296  , 0.08484737, 1.9896222 , ..., 1.5459379 , 0.82842904,
        0.1313783 ],
       [1.0885917 , 0.9853252 , 1.2479072 , ..., 0.28923872, 0.6282486 ,
        0.23339106],
       ...,
       [0.22487839, 2.4103281 , 0.6937058 , ..., 0.403327  , 0.32973105,
        0.9706055 ],
       [0.6627451 , 0.5640983 , 0.6976747 , ..., 0.38385203, 1.6326518 ,
        1.1779834 ],
       [0.02847099, 3.0787446 , 0.28438878, ..., 0.9826051 , 0.01982202,
        1.6046413 ]], dtype=float32), 'vit_feature': array([[ 0.39251244, -0.7359995 , -0.40713686, ...,  1.3599503 ,
        -1.3506224 ,  0.34098613],
       [ 0.536756  ,  0.20046183,  0.53705513, ..., -1.7052754 ,
         0.09141281,  0.14567682],
       [ 0.18865782,  0.40380907,  0.08730512, ...,  1.6099654 ,
         0.558910

In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the features from the dictionary
X_train_resnet = X_train['resnet_feature']  # Choose one feature set, or combine them
X_train_vit = X_train['vit_feature']
X_train_clip = X_train['clip_feature']
X_train_dino = X_train['dino_feature']

# Stack them into one feature matrix (if you want to use all features)
X_train_combined = np.concatenate([X_train_resnet, X_train_vit, X_train_clip, X_train_dino], axis=1)

# Load the labels from the CSV file
labels_df = pd.read_csv('train_labels.csv')  # Assuming the labels are stored in 'labels.csv'
y_train = labels_df['label'].values  # Adjust column name to match your CSV

# Convert labels to integers if they are categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Check the shape of the feature matrix and labels
print("Feature matrix shape:", X_train_combined.shape)
print("Labels shape:", y_train_encoded.shape)


Feature matrix shape: (40000, 2560)
Labels shape: (40000,)


In [38]:
print(f"Shape of ResNet features: {X_train_resnet.shape}")
print(f"Shape of ViT features: {X_train_vit.shape}")
print(f"Shape of CLIP features: {X_train_clip.shape}")
print(f"Shape of DINO features: {X_train_dino.shape}")


Shape of ResNet features: (40000, 512)
Shape of ViT features: (40000, 768)
Shape of CLIP features: (40000, 512)
Shape of DINO features: (40000, 768)


In [39]:
# Normalize the feature values to the range [0, 1]
X_train_combined = X_train_combined / np.max(X_train_combined)

# Split the data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_combined, y_train_encoded, test_size=0.2, random_state=42
)

print("Training set shape:", X_train_split.shape)
print("Validation set shape:", X_val_split.shape)


Training set shape: (32000, 2560)
Validation set shape: (8000, 2560)


In [41]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape

# Define the model
model = Sequential()

# Reshape the input data to (batch_size, 2560, 1)
model.add(Reshape((X_train_combined.shape[1], 1), input_shape=(X_train_combined.shape[1],)))

# Add 1D Convolution layer (1D because data is a 1D feature array)
model.add(Conv1D(32, 3, activation='relu'))  # 32 filters, kernel size 3
model.add(MaxPooling1D(2))  # Max pooling with a pool size of 2

# Flatten the output for fully connected layers
model.add(Flatten())

# Add dense layers for classification
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))  # 10 classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summarize the model architecture
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_1 (Reshape)             │ (None, 2560, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 2558, 32)       │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1279, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 40928)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     5,238,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,240,330 (19.99 MB)

 Trainable params: 5,240,330 (19.99 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# Assuming your labels are in y_train and are integers representing class labels
history = model.fit(X_train_combined, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Optionally, you can evaluate the model on a test dataset
# test_loss, test_accuracy = model.evaluate(X_test, y_test)


Epoch 1/10


I0000 00:00:1733761679.333260     404 service.cc:145] XLA service 0x7f9de8003930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733761679.333553     404 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


 10/500 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.3770 - loss: 2.0753

I0000 00:00:1733761681.226091     404 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.9307 - loss: 0.3262 - val_accuracy: 0.9841 - val_loss: 0.0507
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9885 - loss: 0.0376 - val_accuracy: 0.9827 - val_loss: 0.0557
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9942 - loss: 0.0195 - val_accuracy: 0.9866 - val_loss: 0.0457
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9965 - loss: 0.0109 - val_accuracy: 0.9862 - val_loss: 0.0474
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9983 - loss: 0.0058 - val_accuracy: 0.9852 - val_loss: 0.0556
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9989 - loss: 0.0037 - val_accuracy: 0.9840 - val_loss: 0.0653
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9984 - loss: 0.0058 - val_accuracy: 0.9865 - val_loss: 0.0558
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9989 - loss: 0.0031 - val_accuracy: 0.98

In [43]:
# Load the test data
X_test = np.load('valtest_feats.npy', allow_pickle=True).item()

# Check the structure of the loaded test data to verify it's similar to training data
print(type(X_test))  # Should print <class 'dict'>
print(X_test.keys())  # Should list the keys like 'resnet_feature', 'vit_feature', etc.


<class 'dict'>
dict_keys(['idx', 'resnet_feature', 'vit_feature', 'clip_feature', 'dino_feature'])


In [45]:
# Extract the features from the test data
X_test_resnet = X_test['resnet_feature']
X_test_vit = X_test['vit_feature']
X_test_clip = X_test['clip_feature']
X_test_dino = X_test['dino_feature']

# Combine the features (if necessary, depending on your approach)
# This example assumes you want to concatenate all features together
X_test_combined = np.concatenate([X_test_resnet, X_test_vit, X_test_clip, X_test_dino], axis=1)

# Check the shape of X_test_combined
print(X_test_combined.shape)  # Should print (20000, 2560) (assuming 40000 features from train)


(20000, 2560)


In [46]:
# Predict using the trained model
predictions = model.predict(X_test_combined)

# Convert predictions from one-hot encoded format to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Check the first few predicted labels
print(predicted_labels[:10])  # Print the first 10 predictions


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
[2 0 9 9 8 7 8 5 0 4]


In [47]:
import pandas as pd

# Create a DataFrame with the predictions and ids
result_df = pd.DataFrame({'id': range(len(predicted_labels)), 'label': predicted_labels})

# Save the predictions to a CSV file
result_df.to_csv('result.csv', index=False)
